In [ ]:
#  !conda install -y pillow
#  !conda install -y pandas

In [ ]:
import tensorflow as tf
import pathlib
import numpy as np

In [ ]:
tf.config.list_physical_devices('GPU') 

### Do we need this?

In [ ]:
if True:
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

## Define some base vars.

In [ ]:
img_height = 180
img_width = 320

batch_size = 16


train_path = "data/nuberoja/train"
val_path = "data/nuberoja/validation"


if tf.test.is_built_with_cuda():
    data_format = 'channels_first'
    tf.keras.backend.set_image_data_format('channels_first')
else:
    data_format = 'channels_last'
    tf.keras.backend.set_image_data_format('channels_last')

## Basic autoencoder just to test the dePIXELator

In [ ]:
data_format_shape=(3,img_height,img_width)
if data_format!='channels_first':
    data_format_shape=(img_height,img_width,3)
input_img = tf.keras.layers.Input(shape=data_format_shape)  #source res channels_first for cuda+cudnn

l1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l2(10e-10))(input_img)
l2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l2(10e-10))(l1)
l3 = tf.keras.layers.MaxPool2D(padding='same')(l2)

l4 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l2(10e-10))(l3)
l5 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l2(10e-10))(l4)
l6 = tf.keras.layers.MaxPool2D(padding='same')(l5)

l7 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l2(10e-10))(l6)

l8 = tf.keras.layers.UpSampling2D()(l7)
l9 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l2(10e-10))(l8)
l10 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l2(10e-10))(l9)

l11 = tf.keras.layers.add([l10, l5])

l12 = tf.keras.layers.UpSampling2D()(l11)
l13 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l2(10e-10))(l12)
l14 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l2(10e-10))(l13)

l15 = tf.keras.layers.add([l14, l2])

decoded_image = tf.keras.layers.Conv2D(3, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l2(10e-10))(l15)

auto_encoder = tf.keras.models.Model(inputs=(input_img), outputs=decoded_image)

auto_encoder.compile(optimizer='Adam', loss='mean_squared_error')

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
from batchgenV2 import DePIXELatorBatchGenV2

In [ ]:
data_gen = DePIXELatorBatchGenV2(directory_high_res=train_path,
                 image_size_high=(img_height, img_width),
                 batch_size=batch_size,
                 shuffle=True,
                 seed=42,
                 data_format=data_format)

In [ ]:
validator_gen = DePIXELatorBatchGenV2(directory_high_res=val_path,
                 image_size_high=(img_height, img_width),
                 batch_size=batch_size,
                 shuffle=True,
                 seed=42,
                 data_format=data_format)

In [ ]:
history = auto_encoder.fit(data_gen ,
                epochs=8,
                batch_size=batch_size,
                shuffle=True,
                validation_data=validator_gen,
                callbacks=[early_stop])

## draw history

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
history_df = pd.DataFrame(history.history)
history_df['epoch'] = history.epoch
fig = plt.Figure(figsize=(4, 3))

epochs_to_mean = 1

plt.plot(history_df['loss'].rolling(epochs_to_mean).mean(), 'b')
plt.plot(history_df['val_loss'].rolling(epochs_to_mean).mean(), '-r')
plt.title('Learning Curves')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['traininig loss', 'validation loss'])

## let's see the result:

In [ ]:
testbatch = np.random.randint(0,len(validator_gen))

In [ ]:
decoded_imgs = auto_encoder.predict(validator_gen[testbatch][0])
if data_format=='channels_first':
    decoded_imgs = np.transpose(decoded_imgs, (0, 2, 3, 1))

In [ ]:
n = 4
plt.figure(figsize=(20, n))

for i in range(n):
    # display original
    ax = plt.subplot(2, n, i+1 )
    vraw=validator_gen[testbatch][0]
    if data_format=='channels_first':
        vraw=np.transpose(vraw, (0, 2, 3, 1))
    plt.imshow(vraw[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    # display reconstruction
    ax = plt.subplot(2, n, i + n + 1)
    plt.imshow(decoded_imgs[i].reshape(img_height,img_width,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

## dePIXELize the original frames:

In [ ]:
original_frames_path="data/original_frames"
depixelated_frames_path="data/depixelated"

In [ ]:
import os
for img in os.listdir(original_frames_path ):
        try:
            I = np.asarray(PIL.Image.open(f"{original_frames_path}/{img}").resize((img_width,img_height)) ).astype(np.float32)/255.0
            if data_format=='channels_first':
                image = np.transpose(I, (2, 0, 1))
            res=auto_encoder.predict(image.reshape(-1,3,img_height,img_width))
            res=np.clip(res,0.0,1.0)
            if data_format=='channels_first':
                res = np.transpose(res.reshape((3,img_height,img_width)), (1, 2, 0))
            plt.imsave(f'{depixelated_frames_path}/{img}', res)
            
        except Exception as e:
            print(str(e))
            raise e

## Save model

In [ ]:
model_path = "models/v1"

In [ ]:
#https://machinelearningmastery.com/save-load-keras-deep-learning-models/
# serialize model to JSON
model_json = auto_encoder.to_json()
with open(f"{model_path}/model_{img_width}_{img_height}.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
auto_encoder.save_weights(f"{model_path}/model_{img_width}_{img_height}.h5")
print("Model saved to disk")